# En el casino. Ejemplo de map-reduce

## Generación de apuestas

Generamos apuestas muy simples: dos jugadores lanzan cada uno su dado. Están cargados. Y apuestan una cantidad de dinero.
No nos importa lo que ocurre, pero estas apuestas se recogen en un archivo para su posterior procesamiento estadístico.

In [1]:
import random

dado = ["A", "K", "Q", "J", "8", "7"]

def generar_muestra(n, probs1, probs2, archivo):
    f = open(archivo, "w")
    for i in range(n):
        dado1 = random.choices(dado, probs1)[0]
        dado2 = random.choices(dado, probs2)[0]
        apuesta = 5 * random.randint(1, 100)
        f.write(dado1 + " " + dado2 + " " + str(apuesta) + "\n")
    f.close()

def gana(d1, d2):
    if d1 == d2:
        return random.random() > 0.5
    else:
        return d1.index(d1) >= d2.index(d2)

def ver_inicio_archivo(archivo, n):
    f = open(archivo, "r")
    for i in range(n):
        linea = f.readline().rstrip()
        print(linea)

probs_dado1 = [0.1, 0.3, 0.2, 0.2, 0.1, 0.1]
probs_dado2 = [0.2, 0.2, 0.1, 0.3, 0.1, 0.1]
generar_muestra(10000, probs_dado1, probs_dado2, "casino.txt")
ver_inicio_archivo("casino.txt", 10)

J J 370
J A 100
Q 8 180
K A 295
A K 345
J 8 485
J J 490
8 Q 45
K Q 205
K J 430


## Análisis estadístico de las apuestas

Ahora deseamos examinar el contenido del archivo para ver con qué probabilidades salen los dados.
A diferenica de otros ejemplos con map-reduce, el archivo de datos lo proporcionamos dentro del programa (y no en la línea de comandos), y el resultado se maneja también dentro del programa.

In [11]:
import random
from mrjob.job import MRJob
from collections import defaultdict

def gana(d1, d2):
    if d1 == d2:
        return random.random() > 0.5
    else:
        return d1.index(d1) >= d2.index(d2)

def suma_doble(pares):
    """
    Ejemplo:
    >>> suma_doble([(1, 10), (2, 20), (3, 30)])
    (6, 60)
    """
    a, b = 0, 0
    for x, y in pares:
        a, b = a + x, b + y
    return a, b

class MRComputoApuestas(MRJob):
   
    def mapper(self, _, line):
        linea = line.split()
        dados, apuesta = linea[0] + linea[1], int(linea[2])
        resultado = apuesta if gana(dados[0], dados[1]) else -apuesta
        yield dados, (1, resultado)
              
    def reducer(self, key, values):
        yield key, suma_doble(values)

def demo(nombre_archivo):
    trabajo = MRComputoApuestas(args=[nombre_archivo])
    resultados = []
    with trabajo.make_runner() as runner:
        runner.run()
        for key, value in trabajo.parse_output(runner.cat_output()):
            nuevo_item = key, (value[0], value[1])
            resultados.append(nuevo_item)
    return resultados

def estadisticas(totales):
    frecuencia = defaultdict(lambda:0,{})
    for dados in totales:
        dados[0]
        
"""
resultado = demo("casino.txt")  
total_dados = defaultdict(lambda:0,{})
total_tiradas = 0
for par, totales in resultado:
    dado1, dado2 = par[0], par[1]
    cuantos, total = totales
    total_dados[dado1] += cuantos
    total_dados[dado2] += cuantos
    total_tiradas += cuantos
for d, v in total_dados.items():
    print(d, v/(2*total_tiradas))
"""
trabajo = MRComputoApuestas(args=["casino.txt"])

In [22]:
resultado = trabajo.make_runner().run()

TypeError: expected str, bytes or os.PathLike object, not NoneType